## Tessera analysis of TMAs 1 and 2
##### created on 2025-11-21
##### created by Roman Sankowski

In [103]:
Sys.setenv("PKG_CXXFLAGS" = "-DARMA_64BIT_WORD=1")
options(future.globals.maxSize= 10*1024^3)   # For larger datasets, the memory allowance may need to be increased for parallelization
future::plan(future::multicore)             # Parallelize over multiple samples (if doing multi-sample analysis)

## load functions
source(file.path("R", "functions.R"))

suppressPackageStartupMessages({
    ## Downstream analysis in Seurat V5
    library(Seurat)
    
    ## Plotting functions 
    ## Not imported by Tessera
    library(ggplot2)
    library(ggthemes)
    library(viridis)
    library(patchwork)
    library(tessera)
    library(tidyverse)
    library(Seurat)
    library(harmony)
    library(grafify)
})

In [104]:
#install.packages("grafify")

### Define parameters

In [105]:
fig.size <- function(h, w) {
    options(repr.plot.height = h, repr.plot.width = w)
}


verbose = TRUE
show_plots = TRUE

###### STEP 0 ######
npcs = 10
## Graph pruning
prune_thresh_quantile = 0.95
prune_min_cells = 5


###### STEP 1: GRADIENTS ######
smooth_distance = c('none', 'euclidean', 'projected', 'constant')[3] 
smooth_similarity = c('none', 'euclidean', 'projected', 'constant')[3] 


###### STEP 2: DMT ######
## ... no options


###### STEP 3: AGGREGATION ######
max_npts = 50
min_npts = 5
alpha = 1 ## 0.2 = conservative merging, 2 = liberal merging 

### Load data

In [106]:
obj <- readRDS(file.path("results","merged_nimbus_with_celltypes_codex_object_meningioma.rds"))

## set parameters
fig.size <- function(h, w) {
  options(repr.plot.height = h, repr.plot.width = w)
}

verbose = TRUE
show_plots = TRUE

###### STEP 0 ######
npcs = 5
## Graph pruning
prune_thresh_quantile = 0.95
prune_min_cells = 1


###### STEP 1: GRADIENTS ######
smooth_distance = c('none', 'euclidean', 'projected', 'constant')[3] 
smooth_similarity = c('none', 'euclidean', 'projected', 'constant')[3] 


###### STEP 2: DMT ######
## ... no options


###### STEP 3: AGGREGATION ######
max_npts = 50
min_npts = 5
alpha = 1 ## 0.2 = conservative merging, 2 = liberal merging 


In [107]:
## subset data
obj2 <- obj[,obj$Histology =="Meningioma" & obj$Fixation_protocol=="FFPE"]

## subset counts
counts <- obj2[["RNA"]]$counts

Warning message:
“Adding image data that isn't associated with any assays”
Warning message:
“Not validating Seurat objects”
Warning message:
“Adding image data that isn't associated with any assays”
Warning message:
“Not validating Seurat objects”
Warning message:
“Adding image data that isn't associated with any assays”
Warning message:
“Not validating Seurat objects”
Warning message:
“Adding image data that isn't associated with any assays”
Warning message:
“Not validating Seurat objects”
Warning message:
“Adding image data that isn't associated with any assays”
Warning message:
“Not validating Seurat objects”
Warning message:
“Adding image data that isn't associated with any assays”
Warning message:
“Not validating Seurat objects”
Warning message:
“Adding image data that isn't associated with any assays”
Warning message:
“Not validating Seurat objects”
Warning message:
“Adding image data that isn't associated with any assays”
Warning message:
“Not validating Seurat objects”
Warning 

In [108]:
## get coordinates
## calculate bounding boxes
bboxes <- data.frame(sample=unique(obj2$fov),
                     y=gsub("-.*","",unique(obj2$fov)),
                     x=gsub(".*-","",unique(obj2$fov)))

bboxes$y <- as.numeric(as.factor(bboxes$y))

## add x intercept
bboxes$x_int <- (as.numeric(bboxes$x) - 2) * 4250
bboxes$y_int <- (bboxes$y - 1) * 3680

coords <- map(names(obj2@images), function(x) {
  tryCatch({  
  data.frame(obj2@images[[x]]@coordinates, image=x)
  }, error=function(e){cat("ERROR :",conditionMessage(e), "\n")})
})

coords <- coords %>% 
  bind_rows()
coords2 <- coords %>% 
  mutate(fov=gsub("_.*","",coords$cells)) %>% 
  distinct(fov, image)

rownames(bboxes) <- coords2$image

## re-run coord_extraction with intercepts
coords <- map(names(obj2@images), function(x) {
  tryCatch({  
    .df = obj2@images[[x]]@coordinates
    
    .df$x = .df$x + bboxes[x,]$x_int
    .df$y = .df$y + bboxes[x,]$y_int
    
    .df
    
  }, error=function(e){cat("ERROR :",conditionMessage(e), "\n")})
})

coords <- coords %>% 
  bind_rows()

New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
• `` -> `...3`
New names:
•

In [109]:
meta_data <- data.frame(X= coords[colnames(obj2),]$x,
                        Y= coords[colnames(obj2),]$y,
                        type=obj2$cell_type_2,
                        core=obj2$fov,
                       row.names=colnames(obj2))

In [110]:
## create new seurat object
seurat_obj <- CreateSeuratObject(counts = obj2[["RNA"]]$counts) %>% 
  NormalizeData()

Normalizing layer: counts

Warning message:
“Cannot find a parent environment called Seurat”


In [111]:
# Step 1: Create basic Seurat object
seurat_obj <- CreateSeuratObject(
  counts = counts,
  meta.data = meta_data,
  project = "spatial"
)

# Step 2: Create spatial coordinates matrix
# Seurat expects coordinates in a specific format
coords <- meta_data[, c("X", "Y")]
rownames(coords) <- colnames(counts)  # Must match cell names
colnames(coords) <- c("x", "y")  # Lowercase

# Step 3: Create a FOV (Field of View) object
fov <- CreateFOV(
  coords = coords,
  type = "centroids",  # or "segmentation" if you have boundaries
  molecules = NULL,
  assay = "RNA"
)

# Step 4: Add FOV to Seurat object
seurat_obj[["fov"]] <- fov

In [112]:
# Method 1: Add spatial coordinates as a reduction
coords_matrix <- as.matrix(meta_data[, c("X", "Y")])
rownames(coords_matrix) <- colnames(seurat_obj)
colnames(coords_matrix) <- paste0("spatial_", 1:2)

seurat_obj[["spatial"]] <- CreateDimReducObject(
  embeddings = coords_matrix,
  key = "spatial_",
  assay = "RNA"
)

In [ ]:
## normalize, scale and dimension reduce
VariableFeatures(seurat_obj) = rownames(seurat_obj)[-1]
seurat_obj = seurat_obj |>
    NormalizeData() |>
    ScaleData() |>
    RunPCA() 

# Verify
print(seurat_obj)

Normalizing layer: counts

Centering and scaling data matrix



In [ ]:
seurat_obj@reductions

### Get Tiles

Run the Tessera algorithm to get tiles in one function. The result is returns in two structures: 

1) dmt: cell-level information. 
2) aggs: tile-level information. 

The two are tied together through `dmt$pts$agg_id`


In [ ]:
if (!file.exists(file.path("results","tessera_object_TMA3_24.rds"))) {
    res = GetTiles(
        seurat_obj,        # Single-cell Seurat object
        'spatial',  # Name of dimesional reduction where x/y coordinates are stored
    
        #embeddings = 'harmony',  # (Optional) Name of dimensional reduction where pre-computed single-cell embeddings are stored
        group.by = 'core',  # (Optional) Name of meta.data column that provides sample IDs. If missing, treated as a single sample.
    
        # Additional Tessera algorithm parameters
        prune_thresh_quantile = 0.99, prune_min_cells = 5, # Control pruning of long edges and disconnected cells
        max_npts = 50, min_npts = 5#,                       # Control size of Tessera tiles
       
    )

    ## save object
    saveRDS(res, file.path("results","tessera_object_TMA3_24.rds"))
    
    } else {
    res = readRDS(file.path("results","tessera_object_TMA3_24.rds"))
    }

In [ ]:
## extract the cell and tile matrices
obj = res$obj                # Seurat object of single-cells (with cell-to-tile mapping)
tile_obj = res$tile_obj      # Seurat object of Tessera tiles

In [ ]:
## integrate and cluster tiles
if (!file.exists(file.path("results","tile_object_harmony_clusters.rds"))) {
  tile_obj <- tile_obj %>% 
    NormalizeData() %>% 
    FindVariableFeatures() %>% 
    ScaleData() %>% 
    RunPCA() %>%
    RunHarmony(group.by.vars = "core") %>% 
    RunUMAP(reduction = "harmony", dims=1:30) %>% 
    FindNeighbors(reduction = "harmony", dims=1:30) %>% 
    FindClusters(reduction = "harmony", dims=1:30)
  
  saveRDS(tile_obj, file.path("results","tile_object_harmony_clusters.rds"))
} else {
  tile_obj <- readRDS(file.path("results","tile_object_harmony_clusters.rds"))
}

In [90]:
tail(tile_obj[[]])

,orig.ident,nCount_RNA,nFeature_RNA,id,X,Y,npts,area,perimeter,core,shape,RNA_snn_res.0.8,seurat_clusters
,<fct>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<MULTIPOLYGON>,<fct>,<fct>
O-9_223,SeuratProject,919.2909,68,O-9_223,31505.46,54286.47,34,39942.808,1558.8765,O-9,MULTIPOLYGON (((31576.15 54...,14,14
O-9_224,SeuratProject,766.4346,68,O-9_224,31114.04,54286.42,36,39547.818,1662.2204,O-9,MULTIPOLYGON (((31307.7 544...,4,4
O-9_225,SeuratProject,904.9101,68,O-9_225,31723.99,54335.85,38,48469.445,1293.1885,O-9,MULTIPOLYGON (((31851.15 54...,7,7
O-9_226,SeuratProject,224.7890,68,O-9_226,32156.17,54251.64,8,3552.822,275.7368,O-9,MULTIPOLYGON (((32159.51 54...,2,2
O-9_227,SeuratProject,638.9506,68,O-9_227,31926.49,54307.58,26,15454.858,594.8534,O-9,MULTIPOLYGON (((31851.15 54...,6,6
O-9_228,SeuratProject,135.0756,68,O-9_228,31420.62,54370.13,6,4604.064,322.3457,O-9,MULTIPOLYGON (((31467.72 54...,0,0


In [102]:
head(obj[[]])

,orig.ident,nCount_RNA,nFeature_RNA,X,Y,type,core,tile_id
,<fct>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<fct>
A-10_1,spatial,11.665106,68,35594.05,61.50267,Tumor,A-10,A-10_8
A-10_3,spatial,8.554731,68,35436.36,61.66355,Mural,A-10,A-10_1
A-10_5,spatial,8.245284,68,35659.85,68.47303,Tumor,A-10,A-10_8
A-10_6,spatial,8.096598,68,35541.64,74.56753,P2RY12hi_TAM,A-10,A-10_8
A-10_7,spatial,25.159120,68,35628.68,76.85825,Prolif_TAM,A-10,A-10_8
A-10_8,spatial,12.375905,68,35751.71,78.16667,Mural,A-10,A-10_8


In [92]:
## look at spatial clusters 
table(tile_obj$seurat_clusters)


   0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15 
6841 4903 4588 4342 3386 2594 2388 1839 1624 1611 1536 1243 1167  456  425  366 

In [93]:
## define cell type colors
my_colors <- readRDS(file.path("/Users/romansankowski/Library/CloudStorage/GoogleDrive-romansankowski@gmail.com/Other computers/My MacBook Pro/single_cell_analysis/Visium_HD_meningioma_TMA/results/celltype_colors.rds"))
names(my_colors)[5] <- "Prolif_TAM"
names(my_colors)[10] <- "B_Plasma"
names(my_colors)[8] <- "Tumor"
names(my_colors)[9] <- "Endothelial"
names(my_colors)[20] <- "Mural"
names(my_colors)[6] <- "Mono_DC"
names(my_colors)[4] <- "CD4_T"
names(my_colors)[13] <- "CD8_T"
names(my_colors)[14] <- "other_T"
names(my_colors)[15] <- "Neuroectoderm"
names(my_colors)[16] <- "Proliferating"
names(my_colors)[16] <- "Other_Immune"

In [97]:
## set cluster color
clust_col <- colors_many  #tableau_color_pal(palette = "Hue Circle")(15)
names(clust_col) <- levels(tile_obj$seurat_clusters)
clust_col2 <- grafify:::graf_palettes$kelly
names(clust_col2) <- levels(tile_obj$seurat_clusters)
## reorder tiles
order_tiles <- data.frame(core= tile_obj$core, row.names = rownames(tile_obj[[]])) %>%
  bind_cols(as.data.frame(t(tile_obj[["RNA"]]$scale.data))) %>%
  group_by(core) %>%
  summarize_all(.funs=mean) %>%
  as.data.frame()

rownames(order_tiles) <- order_tiles$core
order_tiles <- order_tiles$core[hclust(dist(order_tiles[,-1]))$order]


ERROR: Error in loadNamespace(x): there is no package called ‘grafify’


In [20]:
## reorder tiles
mat <- data.frame(core= tile_obj$core, row.names = rownames(tile_obj[[]])) %>%
  bind_cols(as.data.frame(t(tile_obj[["RNA"]]$scale.data))) %>%
  group_by(core) %>%
  summarize_all(.funs=mean) %>%
  as.data.frame()

rownames(mat) <- mat$core
order_tiles <- order_tiles[hclust(dist(mat[,-1], method = "euclidean"), method = "complete")$order]

,orig.ident,nCount_RNA,nFeature_RNA,cell,original_cell_id,centroid_x,centroid_y,centroid_z,component,volume,⋯,G2M_Phase_UCell,predicted.id,prediction.score.Tumor,prediction.score.macrophages.DC,prediction.score.Endo.mural,prediction.score.T.NK,prediction.score.B.Plasma,prediction.score.max,celltype,sample_id
,<chr>,<dbl>,<int>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
TMA1_0,SeuratProject,64,34,0,cellid_000044728-1,87.57255,4797.131,0.5,4,21168,⋯,0.000000000,Tumor,1,0,0,0,0,1,Tumor,TMA1
TMA1_1,SeuratProject,297,263,1,cellid_000027998-1,44.42336,2782.093,0.5,4,4380,⋯,0.000000000,Tumor,1,0,0,0,0,1,Tumor,TMA1
TMA1_2,SeuratProject,3436,2168,2,cellid_000027375-1,193.84143,2775.680,0.5,4,21084,⋯,0.002125385,Tumor,1,0,0,0,0,1,Tumor,TMA1
TMA1_3,SeuratProject,6870,3512,3,cellid_000040037-1,122.00403,3000.862,0.5,0,17204,⋯,0.000000000,Tumor,1,0,0,0,0,1,Tumor,TMA1
TMA1_4,SeuratProject,12505,5192,4,cellid_000038378-1,99.82634,3166.368,0.5,0,15328,⋯,0.000000000,Tumor,1,0,0,0,0,1,Tumor,TMA1
TMA1_5,SeuratProject,3661,2161,5,cellid_000048664-1,252.22601,6235.536,0.5,4,49032,⋯,0.006828900,Tumor,1,0,0,0,0,1,Tumor,TMA1


In [89]:
sessionInfo()

R version 4.5.2 (2025-10-31)
Platform: x86_64-apple-darwin13.4.0
Running under: macOS Tahoe 26.1

Matrix products: default
BLAS/LAPACK: /Users/romansankowski/.local/share/mamba/envs/tessera_env/lib/libopenblasp-r0.3.30.dylib;  LAPACK version 3.12.0

locale:
[1] C.UTF-8/UTF-8/C.UTF-8/C/C.UTF-8/C.UTF-8

time zone: Europe/Berlin
tzcode source: system (macOS)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] harmony_1.2.4      lubridate_1.9.4    forcats_1.0.1      stringr_1.6.0     
 [5] dplyr_1.1.4        purrr_1.2.0        readr_2.1.6        tidyr_1.3.1       
 [9] tibble_3.3.0       tidyverse_2.0.0    tessera_0.1.6      Rcpp_1.1.0        
[13] data.table_1.17.8  patchwork_1.3.2    viridis_0.6.5      viridisLite_0.4.2 
[17] ggthemes_5.1.0     ggplot2_4.0.1      Seurat_5.3.1       SeuratObject_5.2.0
[21] sp_2.2-0          

loaded via a namespace (and not attached):
  [1] RColorBrewer_1.1-3     jsonlite_2.0.0 